# Portfolio manipulation

In [132]:
from pathlib import Path
import pandas as pd
import numpy as np

In [78]:
file_name = "template_Portfolio_BILMI.xlsx"
data_path = Path('C:/Users/Hugo Durand/Workspace/data')
file_path = data_path / file_name

In [26]:
portfolio = pd.read_excel(file_path, sheet_name="bil_invest_portfolio")
NAV = pd.read_excel(file_path, sheet_name="NAV")
TPT_base = pd.read_excel(file_path, sheet_name="TPT BIL Invest")
#portfolio.head()
#portfolio.info()

In [22]:
funds = portfolio["FUND NAME"]
funds.drop_duplicates(inplace=True)
#funds
funds.iloc[2]

'BIL INVEST BDS USD CORP INVMT GRADE'

In [27]:
ABSOLUTE_RETURN = portfolio.loc[portfolio["FUND NAME"] == funds.iloc[2]]
#ABSOLUTE_RETURN

# Fetching all "entries" associated to a shareclass 

In [28]:
ISIN = "LU1689732417"

### Find subfund

In [41]:
subfund_code = NAV.loc[NAV["ISIN"] == ISIN]["Subfund_code"]
assert len(subfund_code) == 1
subfund_code = subfund_code.iloc[0]
subfund_code

'LU3964'

### Find all entries associated to subfund

In [178]:
entries = TPT_base.loc[TPT_base["SubFund Code"] == subfund_code]
entries.shape
#entries.info()
entries

,NAV Date,SubFund Code,SubFund Name,SubFund Currency,Portfolio,Accounting Service Code,Accounting Service Name,Valuation Group Code,Valuation Group Label,GTI Code,...,Net Nav Amount,% impact,Unique ID,Multiple Leg ID,Quantity.1,MV inv_ccy,MV fnd_ccy,SC indicator,LU1689729546,Accred expenses splitted
191,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,7.545523611000e+07,-0.002087,294864_1,NaN,-1.574740000000e+03,-1.574740000000e+03,-1.574740000000e+03,P,NaN,NaN
192,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,7.545523611000e+07,-0.003356,290034_1,NaN,-2.532170000000e+03,-2.532170000000e+03,-2.532170000000e+03,P,NaN,NaN
193,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,7.545523611000e+07,-0.012815,290025_1,NaN,-9.669700000000e+03,-9.669700000000e+03,-9.669700000000e+03,IEH,NaN,NaN
194,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,7.545523611000e+07,-0.001065,290018_1,NaN,-8.035100000000e+02,-8.035100000000e+02,-8.035100000000e+02,I,NaN,NaN
195,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,CASH,CO,Debits/credits,850,ACCRUED EXP.,NaN,...,7.545523611000e+07,-0.002234,267100_1,NaN,-1.363500000000e+02,-1.363500000000e+02,-1.363500000000e+02,I,NaN,xxxxxxxxxxxxxx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,201.0,...,7.545523611000e+07,0.266603,US60687YBB48,NaN,2.000000000000e+05,2.012968800000e+05,2.012968800000e+05,NaN,NaN,NaN
354,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,220.0,...,7.545523611000e+07,1.022893,US912810SP49,NaN,8.250000000000e+05,7.762704400000e+05,7.762704400000e+05,NaN,NaN,NaN
355,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,BO,Stock market,150,BONDS,220.0,...,7.545523611000e+07,0.212055,US91282CBD20,NaN,1.600000000000e+05,1.600090100000e+05,1.600090100000e+05,NaN,NaN,NaN
356,2020-12-31,LU3964,BIL INVEST BDS USD CORP INVMT GRADE,USD,SECURITY,FX,Foreign exchange forward,550,F.E.T.,NaN,...,7.545523611000e+07,0.601481,286_1,286.0,4.734002895000e+07,4.734002895000e+07,5.792289242177e+07,IEH,NaN,NaN


### drop entries associated to other shareclasses

In [143]:
# get shareclass indicator
sc_indicator = NAV.loc[NAV.ISIN == ISIN]["SC"]
assert len(sc_indicator) == 1
sc_indicator = sc_indicator.iloc[0]
sc_indicator
# 

'P'

In [144]:
entries = entries.loc[(entries["SC indicator"].isnull()) | (entries["SC indicator"] == sc_indicator)]
entries.shape

(156, 65)

## Compute valuation weight

In [171]:
# not the right way!!!
def compute_valuation_weight(MV, indic):
    if indic == sc_indicator:
        # report value
        return MV / SC_NAV
    else:
        # compute weight
        return (MV / total_MV_fnd_ccy) * (hedged_SC / SC_NAV)

### Compute value for instruments attributed to the shareclass

In [173]:
SC_entries = entries.loc[entries["SC indicator"] == sc_indicator]
#P_entries
SC_NAV = NAV.loc[NAV.ISIN == ISIN]["SC in FUND CCY"].iloc[0]
SC_entries = SC_entries.assign(valuation_weight = SC_entries["MV fnd_ccy"] / SC_NAV)
#P_entries.loc["valuation_weight"]
print(SC_NAV, type(SC_NAV))
print(SC_entries["MV fnd_ccy"].iloc[0], type(SC_entries["MV fnd_ccy"].iloc[0]))
print(SC_entries["valuation_weight"].iloc[0], SC_entries["valuation_weight"].iloc[0])

pd.set_option("display.precision", 12)
SC_entries["valuation_weight"]

11791377.12 <class 'float'>
-1574.74 <class 'numpy.float64'>
-0.00013355013447318188 -0.00013355013447318188


191   -0.000133550134
192   -0.000214747605
201   -0.000976129411
Name: valuation_weight, dtype: float64

###  Compute value for instruments shared over the fund

In [174]:
F_entries = entries.loc[entries["SC indicator"].isnull()]
hedged_SC = NAV.loc[NAV.ISIN == ISIN]["Hedged sc"].iloc[0]
total_MV_fnd_ccy = F_entries["MV fnd_ccy"].sum()
print(total_MV_fnd_ccy)

F_entries = F_entries.assign(valuation_weight = (F_entries["MV fnd_ccy"] / total_MV_fnd_ccy) * (hedged_SC / SC_NAV))
print(F_entries["valuation_weight"].iloc[0])

75066164.82
-0.0031369096659578967
